In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

2023-01-07 23:53:54.414500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 23:53:54.517986: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-07 23:53:54.521769: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-07 23:53:54.521780: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# Define sampling layer

In [2]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z"""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define encoder and decoder

In [3]:
latent_dim = 3
inputlen = 3202
units = 448

# Encoder
encoder_inputs = keras.Input(shape=(inputlen,))
x = layers.Dense(units=units, activation='relu')(encoder_inputs)
x = layers.Dense(units=units//2, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(units=units//2, activation='relu')(latent_inputs)
x = layers.Dense(units=units, activation='relu')(x)
decoder_outputs = layers.Dense(inputlen, activation='linear')(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3202)]       0           []                               
                                                                                                  
 dense (Dense)                  (None, 448)          1434944     ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 224)          100576      ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 3)            675         ['dense_1[0][0]']                
                                                                                            

2023-01-07 23:53:56.369285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-07 23:53:56.369303: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-07 23:53:56.369316: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmhpc): /proc/driver/nvidia/version does not exist
2023-01-07 23:53:56.369469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Define VAE

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #print("reconstruction = " + str(reconstruction))
            #print("z_mean = " + str(z_mean)+"\n")
            #print("z_log_var = " + str(z_log_var)+"\n")
            #print("z = " + str(z)+"\n")
            
            reconstruction_loss = keras.losses.mean_squared_error(data, reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            #print("First kl_loss = "+str(kl_loss)+"\n")
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss,axis=1)) 
            #mprint("Second kl_loss = "+str(kl_loss)+"\n")
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# Load dataset

In [5]:
df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/MotifsDataAutoencoder/Data/3_46cols_EnvFeatures173samples_plus_3202EC_V2_original.csv",sep=",")
df = df.iloc[:,46-len(df.columns):]
print(df.shape)



(173, 3202)


# Load VAE

In [6]:
vae = VAE(encoder, decoder)
vae.load_weights('./vae_checkpoint/my_checkpoint')

# Generate new data

In [7]:
n = 5
scale = 1.0
grid_x = np.linspace(-scale, scale, n)
grid_y = np.linspace(-scale, scale, n)
grid_z = np.linspace(-scale, scale, n)

for i, xi in enumerate(grid_x):
    for j, yi in enumerate(grid_y):
        for k, zi in enumerate(grid_z): 
            z_sample = np.array([[xi, yi, zi]])
            decoded = vae.decoder.predict(z_sample)
            #print(z_sample)
            print(decoded)
            print(decoded.shape)
            #print("")

1/1 [==============================] - 0s 85ms/step
[[8.2116649e-02 1.2414937e-03 2.4878459e-02 ... 1.7432263e-04
  9.4860554e-02 2.8447106e-01]]
(1, 3202)
1/1 [==============================] - 0s 14ms/step
[[8.2211226e-02 1.2840175e-03 2.4956094e-02 ... 1.8699002e-04
  9.4981179e-02 2.8448933e-01]]
(1, 3202)
1/1 [==============================] - 0s 14ms/step
[[8.2297683e-02 1.3258292e-03 2.5032431e-02 ... 2.0471890e-04
  9.5096074e-02 2.8451598e-01]]
(1, 3202)
1/1 [==============================] - 0s 15ms/step
[[8.2374752e-02 1.3668085e-03 2.5107265e-02 ... 2.2826158e-04
  9.5204346e-02 2.8455180e-01]]
(1, 3202)
1/1 [==============================] - 0s 15ms/step
[[8.2451835e-02 1.4077898e-03 2.5182098e-02 ... 2.5179586e-04
  9.5312648e-02 2.8458774e-01]]
(1, 3202)
1/1 [==============================] - 0s 19ms/step
[[8.2118519e-02 1.2593467e-03 2.4896730e-02 ... 1.6055955e-04
  9.4694838e-02 2.8443909e-01]]
(1, 3202)
1/1 [==============================] - 0s 13ms/step
[[8.2213111e